In [ ]:
pip install -U  finance-datareader # type: ignore

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # pandas 명시적 import 필요
import FinanceDataReader as fdr # type: ignore
import random
from tqdm import tqdm

# 우리가 만든 모듈들 임포트
# (data_process.py에 clean_price_data, log_data가 있다고 가정)
from base.data_process import * 
from diffusion_config import DiffusionConfig
from diffusion_factor_model import Encoder, Decoder
from time_dynamics import TemporalDynamics
from base.noise import GaussianDiffusion
from base.pca import PCA
from base.loss_functions import MSE_Loss

# ---------------------------------------------------------
# 1. 설정 및 장치 초기화
# ---------------------------------------------------------
config = DiffusionConfig()

# [안전장치] Config에 device가 없으면 여기서 직접 설정
if hasattr(config, 'device'):
    device = config.device
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device set to: {device}")


config_dict = config.to_dict()

In [ ]:
# ---------------------------------------------------------
# 2. 데이터 다운로드 설정
# ---------------------------------------------------------
# 일단 목표 개수는 config에서 가져오되, 나중에 업데이트할 예정
TARGET_N_STOCKS = config.num_assets 
START_DATE = '2015-01-01'
END_DATE = '2025-12-31'

# ---------------------------------------------------------
# 3. S&P 500 종목 리스트 가져오기
# ---------------------------------------------------------
print("S&P 500 종목 리스트를 불러오는 중...")
try:
    sp500 = fdr.StockListing('S&P500')
    all_tickers = sp500['Symbol'].tolist()
    print(f"총 {len(all_tickers)}개의 종목을 찾았습니다.")
except Exception as e:
    print(f"S&P500 리스트 로드 실패 (인터넷 연결 확인 필요): {e}")
    # 테스트용 더미 리스트 (실패 시 비상용)
    all_tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NVDA', 'TSLA', 'META', 'BRK-B', 'LLY', 'V']

# ---------------------------------------------------------
# 4. 랜덤하게 n개 뽑기
# ---------------------------------------------------------
if TARGET_N_STOCKS > len(all_tickers):
    selected_tickers = all_tickers
else:
    selected_tickers = random.sample(all_tickers, TARGET_N_STOCKS)

print(f"다운로드 시도할 종목 수: {len(selected_tickers)}")
print(f"예시: {selected_tickers[:5]} ...")

# ---------------------------------------------------------
# 5. 데이터 다운로드 및 병합
# ---------------------------------------------------------
close_data_dict = {}

print("데이터 다운로드 시작...")
for ticker in tqdm(selected_tickers):
    try:
        df = fdr.DataReader(ticker, START_DATE, END_DATE)
        
        # 데이터가 있고, 결측치가 너무 많지 않은 경우에만 추가
        if not df.empty and 'Close' in df.columns:
            close_data_dict[ticker] = df['Close']
            
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        continue

# ---------------------------------------------------------
# 6. DataFrame 생성 및 전처리
# ---------------------------------------------------------
price_df = pd.DataFrame(close_data_dict)

# [중요] 전처리 함수들이 정의되어 있어야 합니다.
# 만약 import * 로 안 불러와졌다면 여기서 정의하거나 data_process.py 확인 필요
price_df = clean_price_data(price_df)

# 로그 수익률 변환
ln_price, ln_price_diff = log_data(price_df)

# 추가 정제
ln_price = clean_price_data(ln_price)
ln_price_diff = clean_price_data(ln_price_diff)

# ---------------------------------------------------------
# 7. [CRITICAL] Config 업데이트 (실제 데이터 개수에 맞춤)
# ---------------------------------------------------------
actual_n_stocks = ln_price_diff.shape[1]

if actual_n_stocks != config.num_assets:
    print(f"\n[주의] 요청한 종목 수({config.num_assets})와 다운로드 성공한 수({actual_n_stocks})가 다릅니다.")
    print(f"Config.num_assets를 {actual_n_stocks}로 자동 업데이트합니다.")
    config.num_assets = actual_n_stocks
    
    # [Tip] 만약 num_factors가 종목 수보다 크면 에러 나므로 체크
    if config.num_factors > actual_n_stocks:
        print(f"Warning: 팩터 개수({config.num_factors})가 종목 수보다 많습니다. 절반으로 조정합니다.")
        config.num_factors = actual_n_stocks // 2

# ---------------------------------------------------------
# 8. 최종 확인
# ---------------------------------------------------------
print("-" * 50)
print(f"[완료] 데이터프레임 생성됨.")
print(f"Price Data Shape: {ln_price.shape} (Row: 날짜, Col: 종목)")
print(f"Diff Data Shape: {ln_price_diff.shape} (Input Data)")
print(f"Config Num Assets: {config.num_assets}")
print("-" * 50)

# 결과 확인
ln_price_diff.head()

In [ ]:
# ---------------------------------------------------------
# 1. 데이터 전처리 (To_TensorSet 실행)
# ---------------------------------------------------------
print("데이터 텐서 변환 및 분할 시작...")

# (1) 프로세서 인스턴스 생성 (Config 필요)
tensor_processor = To_TensorSet(config)

# (2) process 메서드 호출 (DataFrame -> Dictionary)
# diffusion_data_dict['train']['FULL']에 (N, d, T) 형태의 텐서가 들어있습니다.
diffusion_data_dict = tensor_processor.process(ln_price_diff)

print("텐서 변환 완료.")



In [ ]:
# ---------------------------------------------------------
# 2. PCA 수행 (Train Set 전체 데이터 사용)
# ---------------------------------------------------------
# PCA는 학습 데이터 전체의 분포를 봐야 하므로 배치로 나누기 전의 'FULL' 데이터를 사용합니다.
train_full_data = diffusion_data_dict['train']['FULL']

# GPU 사용 가능하다면 GPU로 이동 (연산 가속)
train_full_data = train_full_data.to(device)

print(f"\nPCA 분석 시작... (Input Shape: {train_full_data.shape})")

# PCA 함수 호출
# k = config.num_factors (예: 10개 팩터)
cov_full, eig_vals, diag_variances = PCA(train_full_data, k=config.num_factors)

# ---------------------------------------------------------
# 3. 결과 저장 및 확인
# ---------------------------------------------------------
# [중요] diag_variances는 나중에 Loss Function에서 가중치로 계속 쓰이므로
# 미리 device에 올려두고 변수로 저장해둡니다.
diag_variances = diag_variances.to(device)

print("-" * 50)
print(f"PCA 완료. 추출된 잔차 분산(Variance) 개수: {len(diag_variances)}")
print(f"상위 5개 고유값(Eigenvalues): {eig_vals[:5].cpu().numpy()}") # 확인용 출력
print(f"첫 5개 종목의 잔차 분산: {diag_variances[:5].cpu().numpy()}")
print("-" * 50)

# 시각화 (선택 사항: 고유값 감소 그래프 Scree Plot)


plt.figure(figsize=(10, 5))
plt.plot(eig_vals.cpu().numpy(), marker='o')
plt.title("Scree Plot (Eigenvalues)")
plt.xlabel("Factor Index")
plt.ylabel("Eigenvalue Size")
plt.grid(True)
plt.show() #

In [ ]:
import itertools

#---------------------------------------------------------
# 1. 모델 및 설정 초기화 (Setup)
# ---------------------------------------------------------
print("모델 초기화 및 GPU 설정 중...")

# (1) 설정 및 장치
# config는 앞서 정의한 DiffusionConfig 객체여야 합니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current Device: {device}")

# (2) 모델 인스턴스 생성
# Encoder: (Batch, D, T) -> (Batch, k, T)
encoder = Encoder(input_dim=config.num_assets, 
                  factor_dim=config.num_factors, 
                  num_layers=2).to(device)

# Noise Generator: (Batch, k, T) -> (Batch, k, T)
# (파라미터 학습 X, 버퍼만 있음)
diffusion = GaussianDiffusion(config).to(device)

# Time Dynamics: (Batch, k, T) + t -> (Batch, k, T)
dynamics = TemporalDynamics(factor_dim=config.num_factors, 
                            time_emb_dim=config.hidden_dim, 
                            num_layers=4).to(device)

# Decoder: (Batch, k, T) -> (Batch, D, T)
decoder = Decoder(factor_dim=config.num_factors, 
                  output_dim=config.num_assets, 
                  num_layers=2).to(device)

# (3) Optimizer 설정

all_parameters = list(itertools.chain(
    encoder.parameters(),
    dynamics.parameters(),
    decoder.parameters()
))

optimizer = optim.AdamW(all_parameters, lr=1e-3, weight_decay=1e-5)

# (4) Loss Function 설정
# use_cov_weight=True -> PCA 잔차 분산 사용
loss_fn_R = MSE_Loss(use_cov_weight=True, prepared_var_mtx=True).to(device)
loss_fn_F = MSE_Loss(use_cov_weight=False).to(device) # 팩터는 단순 MSE

# PCA 결과인 잔차 분산을 GPU로 이동 (Loss 계산용)
diag_variances = diag_variances.to(device)

# ---------------------------------------------------------
# 2. 학습 루프 (Training Loop)
# ---------------------------------------------------------
print("\n학습 시작...")

# 데이터 로더 (List of Tensors from To_TensorSet)
train_loader = diffusion_data_dict['train']['BDT'] # (Batch, D, T) 형태

EPOCHS = config.num_epochs
loss_history = []

for epoch in range(EPOCHS):
    epoch_loss_total = 0.0
    epoch_loss_r = 0.0
    epoch_loss_f = 0.0
    
    # --- Batch Loop ---
    for batch_idx, real_data in enumerate(train_loader):
        # 1. 데이터 준비
        # real_data: (Batch, D, T) -> GPU 이동
        x_0 = real_data.to(device)
        batch_size = x_0.size(0)
        
        # Optimizer 초기화
        optimizer.zero_grad()
        
        # -------------------------------------------------
        # [Step 1] Encoder: Clean Data -> Clean Factor
        # -------------------------------------------------
        f_0 = encoder(x_0) # (B, k, T)
        
        # -------------------------------------------------
        # [Step 2] Diffusion: Clean Factor -> Noisy Factor
        # -------------------------------------------------
        # f_0에 노이즈 주입 (학습용 문제 출제)
        f_t, noise_z, t = diffusion(f_0) 
        
        # -------------------------------------------------
        # [Step 3] Dynamics: Noisy Factor -> Predicted Factor
        # -------------------------------------------------
        # "노이즈 낀 f_t를 보고 원본 f_0를 맞춰봐"
        f_0_hat = dynamics(f_t, t)
        
        # -------------------------------------------------
        # [Step 4] Decoder: Predicted Factor -> Reconstructed Data
        # -------------------------------------------------
        # "복원된 팩터로 자산 가격 다시 그려봐"
        r_0_hat = decoder(f_0_hat)
        
        # -------------------------------------------------
        # [Step 5] Loss Calculation
        # -------------------------------------------------
        
        # (A) Reconstruction Loss (자산 복원)
        # PCA 잔차 분산(diag_variances)을 가중치로 사용
        loss_R = loss_fn_R(r_0_hat, x_0, var_mtx=diag_variances)
        
        # (B) Latent Consistency Loss (팩터 일관성)
        # Encoder가 만든 정답(f_0) vs Dynamics가 예측한 값(f_0_hat)
        loss_F = loss_fn_F(f_0_hat, f_0)
        
        # (C) Total Loss
        # lambda_f = 0.1 (팩터 로스 반영 비율, 조절 가능)
        lambda_f = 0.1
        total_loss = loss_R + (lambda_f * loss_F)
        
        # -------------------------------------------------
        # [Step 6] Backpropagation
        # -------------------------------------------------
        total_loss.backward()
        
        # Gradient Clipping (학습 안정성)
        torch.nn.utils.clip_grad_norm_(all_parameters, max_norm=1.0)
        
        optimizer.step()
        
        # 로그 기록
        epoch_loss_total += total_loss.item()
        epoch_loss_r += loss_R.item()
        epoch_loss_f += loss_F.item()

    # --- Epoch End ---
    # 평균 Loss 계산
    avg_total = epoch_loss_total / len(train_loader)
    avg_r = epoch_loss_r / len(train_loader)
    avg_f = epoch_loss_f / len(train_loader)
    loss_history.append(avg_total)
    
    if (epoch + 1) % 10 == 0:
        print(f"[Epoch {epoch+1}/{EPOCHS}] "
              f"Total: {avg_total:.6f} | Recon(R): {avg_r:.6f} | Latent(F): {avg_f:.6f}")

print("학습 완료.")

# ---------------------------------------------------------
# 3. 학습 결과 시각화 (Loss Curve)
# ---------------------------------------------------------
plt.figure(figsize=(10, 5))
plt.plot(loss_history, label='Total Loss')
plt.title("Training Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show() 